In [1]:
import pandas as pd
from ydata_profiling import ProfileReport
df = pd.read_csv("base_trie.csv")

In [3]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797885 entries, 0 to 797884
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      797885 non-null  object 
 1   StockCode    797885 non-null  object 
 2   Description  797885 non-null  object 
 3   Quantity     797885 non-null  int64  
 4   InvoiceDate  797885 non-null  object 
 5   Price        797885 non-null  float64
 6   Customer ID  797885 non-null  float64
 7   Country      797885 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 48.7+ MB


### Engin Featuring.

### Country amount

In [12]:
import pandas as pd

# Charger le dataset
df = pd.read_csv('base_trie.csv')

# Calculer le montant total des achats par client
df['TotalAmount'] = df['Quantity'] * df['Price']

# Regrouper par client et pays, puis calculer la somme totale des montants
df_country_amount = df.groupby(['Customer ID', 'Country'])['TotalAmount'].sum().unstack().reset_index()

# Remplacer les valeurs manquantes par 0 (pour les clients n'ayant pas dépensé dans un pays spécifique)
df_country_amount.fillna(0, inplace=True)

# Renommer les colonnes des pays pour refléter l'information
df_country_amount.rename(columns={'Country': 'Customer ID'}, inplace=True)

# Afficher les premières lignes du résultat
print(df_country_amount.head())


Country  Customer ID  Australia  Austria  Bahrain  Belgium  Brazil  Canada  \
0            12346.0        0.0      0.0      0.0      0.0     0.0     0.0   
1            12347.0        0.0      0.0      0.0      0.0     0.0     0.0   
2            12348.0        0.0      0.0      0.0      0.0     0.0     0.0   
3            12349.0        0.0      0.0      0.0      0.0     0.0     0.0   
4            12350.0        0.0      0.0      0.0      0.0     0.0     0.0   

Country  Channel Islands  Cyprus  Czech Republic  ...  Singapore  Spain  \
0                    0.0     0.0             0.0  ...        0.0    0.0   
1                    0.0     0.0             0.0  ...        0.0    0.0   
2                    0.0     0.0             0.0  ...        0.0    0.0   
3                    0.0     0.0             0.0  ...        0.0    0.0   
4                    0.0     0.0             0.0  ...        0.0    0.0   

Country  Sweden  Switzerland  Thailand  USA  United Arab Emirates  \
0          

In [20]:
df_country_amount.shape

(5942, 42)

### Time Diff

In [14]:
import pandas as pd

# Charger le dataset
df = pd.read_csv('base_trie.csv')

# Convertir la colonne "InvoiceDate" en type de données datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Trier le DataFrame par client et date
df_sorted = df.sort_values(by=['Customer ID', 'InvoiceDate'])

# Calculer la différence de jours entre les achats pour chaque client
df_sorted['TimeDiff'] = df_sorted.groupby('Customer ID')['InvoiceDate'].diff().dt.days

# Calculer la fréquence d'achat (nombre moyen de jours entre les achats) pour chaque client
df_frequency = df_sorted.groupby('Customer ID')['TimeDiff'].mean().reset_index()

# Afficher les premières lignes du résultat
print(df_frequency.head())


   Customer ID  TimeDiff
0      12346.0  8.543478
1      12347.0  1.805430
2      12348.0  7.240000
3      12349.0  3.994413
4      12350.0  0.000000


In [15]:
df_frequency.shape

(5942, 2)

### Total Amount

In [8]:
import pandas as pd

# Charger le dataset
df = pd.read_csv('base_trie.csv')

# Calculer le montant total des achats par client
df['TotalAmount'] = df['Quantity'] * df['Price']

# Regrouper par client et calculer la somme totale des montants
df_total_amount = df.groupby('Customer ID')['TotalAmount'].sum().reset_index()

# Afficher les premières lignes du résultat
print(df_total_amount.head())


   Customer ID  TotalAmount
0      12346.0       -51.74
1      12347.0      4921.53
2      12348.0      2019.40
3      12349.0      4404.54
4      12350.0       334.40


In [17]:
df_total_amount.shape

(5942, 2)

### Moyen achat

In [18]:
import pandas as pd

# Charger le dataset
df = pd.read_csv('base_trie.csv')

# Calculer le montant total des achats par transaction
df['TotalAmount'] = df['Quantity'] * df['Price']

# Regrouper par client et calculer le montant moyen des achats
df_avg_amount = df.groupby('Customer ID')['TotalAmount'].mean().reset_index()

# Renommer la colonne pour refléter l'information
df_avg_amount.rename(columns={'TotalAmount': 'AverageAmount'}, inplace=True)

# Afficher les premières lignes du résultat
print(df_avg_amount.head())


   Customer ID  AverageAmount
0      12346.0      -1.100851
1      12347.0      22.169054
2      12348.0      39.596078
3      12349.0      24.469667
4      12350.0      19.670588


In [19]:
df_avg_amount.shape

(5942, 2)

In [7]:
df['Description'].nunique()

5299

### Merge

In [21]:
import pandas as pd

# Fusionner les dataframes en utilisant la colonne "Customer ID" comme clé de fusion
merged_df = pd.merge(df_avg_amount, df_total_amount, on='Customer ID', how='inner')
merged_df = pd.merge(merged_df, df_frequency, on='Customer ID', how='inner')
merged_df = pd.merge(merged_df, df_country_amount, on='Customer ID', how='inner')

# Afficher le dataframe fusionné
print(merged_df.head())


   Customer ID  AverageAmount  TotalAmount  TimeDiff  Australia  Austria  \
0      12346.0      -1.100851       -51.74  8.543478        0.0      0.0   
1      12347.0      22.169054      4921.53  1.805430        0.0      0.0   
2      12348.0      39.596078      2019.40  7.240000        0.0      0.0   
3      12349.0      24.469667      4404.54  3.994413        0.0      0.0   
4      12350.0      19.670588       334.40  0.000000        0.0      0.0   

   Bahrain  Belgium  Brazil  Canada  ...  Singapore  Spain  Sweden  \
0      0.0      0.0     0.0     0.0  ...        0.0    0.0     0.0   
1      0.0      0.0     0.0     0.0  ...        0.0    0.0     0.0   
2      0.0      0.0     0.0     0.0  ...        0.0    0.0     0.0   
3      0.0      0.0     0.0     0.0  ...        0.0    0.0     0.0   
4      0.0      0.0     0.0     0.0  ...        0.0    0.0     0.0   

   Switzerland  Thailand  USA  United Arab Emirates  United Kingdom  \
0          0.0       0.0  0.0                   0.0

In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5942 entries, 0 to 5941
Data columns (total 45 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Customer ID           5942 non-null   float64
 1   AverageAmount         5942 non-null   float64
 2   TotalAmount           5942 non-null   float64
 3   TimeDiff              5791 non-null   float64
 4   Australia             5942 non-null   float64
 5   Austria               5942 non-null   float64
 6   Bahrain               5942 non-null   float64
 7   Belgium               5942 non-null   float64
 8   Brazil                5942 non-null   float64
 9   Canada                5942 non-null   float64
 10  Channel Islands       5942 non-null   float64
 11  Cyprus                5942 non-null   float64
 12  Czech Republic        5942 non-null   float64
 13  Denmark               5942 non-null   float64
 14  EIRE                  5942 non-null   float64
 15  European Community   

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797885 entries, 0 to 797884
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      797885 non-null  object 
 1   StockCode    797885 non-null  object 
 2   Description  797885 non-null  object 
 3   Quantity     797885 non-null  int64  
 4   InvoiceDate  797885 non-null  object 
 5   Price        797885 non-null  float64
 6   Customer ID  797885 non-null  float64
 7   Country      797885 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 48.7+ MB


### Score RFM

In [39]:
import pandas as pd

# Charger le dataset
df = pd.read_csv('base_trie.csv')

# Convertir la colonne "InvoiceDate" en type de données datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Calculer la date la plus récente dans le dataset
most_recent_date = df['InvoiceDate'].max()

# Calculer la récence (Recency) en jours
df['Recency'] = (most_recent_date - df.groupby('Customer ID')['InvoiceDate'].transform('max')).dt.days

# Calculer la fréquence (Frequency)
df['Frequency'] = df.groupby('Customer ID')['Invoice'].transform('nunique')

# Calculer le montant (Monetary)
df['Monetary'] = df.groupby('Customer ID')['Price'].transform('sum')

# Créer une fonction pour attribuer des scores RFM
def assign_rfm_score(value, quantiles):
    if value <= quantiles[0]:
        return 4
    elif value <= quantiles[1]:
        return 3
    elif value <= quantiles[2]:
        return 2
    else:
        return 1

# Calculer les quantiles pour chaque mesure RFM
recency_quantiles = df['Recency'].quantile(q=[0.25, 0.5, 0.75]).values
frequency_quantiles = df['Frequency'].quantile(q=[0.25, 0.5, 0.75]).values
monetary_quantiles = df['Monetary'].quantile(q=[0.25, 0.5, 0.75]).values

# Appliquer la fonction d'assignation des scores RFM pour chaque mesure
df['R'] = df['Recency'].apply(assign_rfm_score, args=(recency_quantiles,))
df['F'] = df['Frequency'].apply(assign_rfm_score, args=(frequency_quantiles,))
df['M'] = df['Monetary'].apply(assign_rfm_score, args=(monetary_quantiles,))

# Concaténer les scores R, F, M pour créer l'indice RFM
df['RFM'] = df['R'].astype(str) + df['F'].astype(str) + df['M'].astype(str)

# Afficher le dataframe avec l'indice RFM
print(df.head())



  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  Recency  Frequency  \
0 2009-12-01 07:45:00   6.95      13085.0  United Kingdom      157         10   
1 2009-12-01 07:45:00   6.75      13085.0  United Kingdom      157         10   
2 2009-12-01 07:45:00   6.75      13085.0  United Kingdom      157         10   
3 2009-12-01 07:45:00   2.10      13085.0  United Kingdom      157         10   
4 2009-12-01 07:45:00   1.25      13085.0  United Kingdom      157         10   

   Monetary  R  F  M  RFM  
0   1142.05  1  3  2  132  
1   1142.05  1  3  2  132  
2   1142

In [40]:
df['RFM'] = df['RFM'].astype(int)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797885 entries, 0 to 797884
Data columns (total 15 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      797885 non-null  object        
 1   StockCode    797885 non-null  object        
 2   Description  797885 non-null  object        
 3   Quantity     797885 non-null  int64         
 4   InvoiceDate  797885 non-null  datetime64[ns]
 5   Price        797885 non-null  float64       
 6   Customer ID  797885 non-null  float64       
 7   Country      797885 non-null  object        
 8   Recency      797885 non-null  int64         
 9   Frequency    797885 non-null  int64         
 10  Monetary     797885 non-null  float64       
 11  R            797885 non-null  int64         
 12  F            797885 non-null  int64         
 13  M            797885 non-null  int64         
 14  RFM          797885 non-null  int64         
dtypes: datetime64[ns](1), float64(3), 